# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
# Import API key
from api_keys import g_key

In [2]:
# Store Part I results into DataFrame


In [20]:

weather_df = pd.read_csv("../output_data/weather_df.csv")
weather_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Guhāgar,IN,1597863176,17.48,73.19,78.71,88,100,10.20
1,1,Jamestown,US,1597863341,42.10,-79.24,69.01,55,40,9.17
2,2,Talnakh,RU,1597863341,69.49,88.40,55.40,54,0,8.95
3,3,Albany,US,1597862517,42.60,-73.97,75.99,53,75,9.17
4,4,Kapaa,US,1597863342,22.08,-159.32,80.01,78,75,11.41


In [21]:
weather_df.describe()

,City_ID,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
count,559.000000,5.590000e+02,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000
mean,279.000000,1.597863e+09,20.331431,20.450716,68.652952,68.776386,48.912343,8.141556
std,161.513673,9.480420e+01,33.346833,88.611595,15.030822,21.947602,39.977070,5.223078
min,0.000000,1.597863e+09,-54.800000,-175.200000,30.610000,9.000000,0.000000,0.290000
25%,139.500000,1.597863e+09,-6.520000,-55.265000,55.860000,56.500000,3.000000,4.560000
50%,279.000000,1.597863e+09,23.920000,24.480000,71.550000,74.000000,40.000000,6.930000
75%,418.500000,1.597863e+09,48.330000,100.345000,79.260000,85.000000,90.000000,10.640000
max,558.000000,1.597863e+09,78.220000,179.320000,106.610000,100.000000,100.000000,36.910000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

In [17]:
locations = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
fig = gmaps.figure(layout={'width': '800px', 'height': '600px'})
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=300, 
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='600px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


good_weather_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                 (weather_df["Max Temperature"] > 70) & (weather_df["Max Temperature"] < 80)].dropna()

good_weather_df.head()



,City_ID,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
6,6,Goderich,CA,1597863343,43.75,-81.72,72.00,64,0,5.99
171,171,Mehrān,IR,1597863385,33.12,46.16,78.80,19,0,6.93
189,189,Kamina,CD,1597863390,-8.74,24.99,74.95,25,0,3.98
287,287,Ogulin,HR,1597863412,45.27,15.23,75.20,64,0,1.12
323,323,Çan,TR,1597863420,40.03,27.05,78.80,73,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = good_weather_df[["City","Country", "Longitude", "Latitude"]].copy()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Longitude,Latitude,Hotel Name
6,Goderich,CA,-81.72,43.75,
171,Mehrān,IR,46.16,33.12,
189,Kamina,CD,24.99,-8.74,
287,Ogulin,HR,15.23,45.27,
323,Çan,TR,27.05,40.03,
355,Naze,JP,129.48,28.37,
362,Salaspils,LV,24.37,56.86,
391,Rassvet,RU,40.75,46.83,
476,Mola di Bari,IT,17.09,41.06,
523,Poum,NC,164.02,-20.23,


In [33]:
list_hotels = []

for index, row in hotel_df.iterrows():
    
    lat = row['Latitude']
    lng = row['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
               
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        list_hotels.append(jsn['results'][0]['name'])
    except:
        list_hotels.append("")
        pass
            
hotel_df["Hotel Name"] = list_hotels
#good_weather_df = good_weather_df.dropna(how='any')
hotel_df.head()

,City,Country,Longitude,Latitude,Hotel Name
6,Goderich,CA,-81.72,43.75,Harmony Inn
171,Mehrān,IR,46.16,33.12,بوتیک زنانه بهار
189,Kamina,CD,24.99,-8.74,Guest House la Grace
287,Ogulin,HR,15.23,45.27,Hotel Frankopan
323,Çan,TR,27.05,40.03,Kardeş Hotel


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='800px'))